# Import Models as Python Functions

A useful feature of yggdrasil is that you can use the mechanics to call models as functions from inside Python even if the model is not written in Python. For example, the model displayed below is written in Fortran, but can be called from Python via the yggdrasil `import_as_function` method.

In [ ]:
from yggdrasil import tools
tools.display_source('models/light_v0.f90', number_lines=True)
tools.display_source('yamls/light_v0_fortran.yml', number_lines=True)

When import_as_function is called, the model yaml is loaded and models contained in the yaml are run on forked processes after being copiled as necessary. 

In [ ]:
from yggdrasil import import_as_function
light = import_as_function('yamls/light_v0_fortran.yml')

The returned function's inputs & outputs are determined by the unpaired inputs/outputs located in the yaml. In this example, the only inputs & outputs come from the server.

In [ ]:
light.model_info()

The light model can then be called as a function by providing input arguments. If the values provided do not have any units, the default units expected by the light model are assumed (i.e. days and cm).

In [ ]:
print(light(100.0, 100.0))
print(light(1.0, 2.9))
print(light(2.0, 3.0))

If units are added, the values will be converted into the units expected by the model.

In [ ]:
from yggdrasil import units
print(light(units.add_units(24.0, 'hrs'), units.add_units(2.9, 'cm')))
print(light(units.add_units(1.0, 'days'), units.add_units(0.029, 'm')))

In [ ]:
light.stop()

## <span style="color:#C52060">Test your knowledge #7</span>

1. <span style="color:#C52060">Try importing the `models/weather.py` model in the cell below. Depending on how you wrote it, you may have to modify the YAML so that the model is alone in a file (i.e. no other models or connections).</span>
1. <span style="color:#C52060">Try importing the `models/co2.py` model in the cell below.</span>

##### Tip: If a model receives or send multiple variables from/to a channel, those variables will need to be explicitly named to be passed separately to/from the imported function

# Overhead
Although yggdrasil makes it easy to connect models, the connections come with a performance overhead and can never match the performance from passing messages directly within a language. For example, the three cells below make the same calculation (sine), but the first calls a Python method direction, the second calls the same Python method but via yggdrasil, and the third calls the Fortran `SIN` function via yggdrasil. 

The direct call to the Python method (first cell) is much more efficient. Similarly, if two models are written in the same language, it will be much more preformant to call the code directly (unless the communication pattern allows models to make computations in parallel).

In [ ]:
# Calculation using numpy in Python
import numpy as np
%timeit np.sin(0.0)

In [ ]:
# Calculation calling numpy in Python via yggdrasil
sine_python = import_as_function('yamls/sine_model_python.yml')
%timeit sine_python(0.0)
sine_python.stop()

In [ ]:
# Calculation calling the fortran SIN function via yggdrasil
sine_fortran = import_as_function('yamls/sine_model_fortran.yml')
%timeit sine_fortran(0.0)
sine_fortran.stop()

# Models built by make/cmake
Many of the existing computational models contain a large amount of source code organized in directories, modules, etc. For models written in compiled languages, this size and complexity means it is often necessary to use a build tool to compile the model. Yggdrasil allows for such cases by supporting `make` and `cmake` model language parameters.

For example, the cell below displays three YAML files. The first is the YAML required to run a C version of the `timesync` enabled roots model from the previous notebook. The second is the YAML required to run the same code, but compiled via make. The third uses cmake. 

Here yggdrasil assumes the standard build file name (`Makefile` and `CMakeLists.txt`), but the name can also be passed explicitly. Yggdrasil adds the compilation and linking flags to the build by either adjusting the environment variables used by most builds (as in the case of `make`) or creating a dummy version of the build file that adds these flags to the build rules for the selected target (as in the cas of `cmake`).

In [ ]:
tools.display_source('yamls/roots_v1_c.yml', number_lines=True)
tools.display_source_diff('yamls/roots_v1_c.yml', 'yamls/roots_v1_make.yml', number_lines=True)
tools.display_source_diff('yamls/roots_v1_c.yml', 'yamls/roots_v1_cmake.yml', number_lines=True)

# Transforming Inputs & Outputs
Sometimes you need to make modifications to the input/output in a connection either based on the problem you are solving or in order to make them compatible. One option is to handle these modifications in the model, but if you are wrapping an existing function or want to avoid modify the model source code. For such cases, yggdrasil provides methods of transforming data through specification in the yaml.

The cell below shows the diff for a set of connections to the light model that transforms the intensities from the light model before passing them to the output file by multiplying it by a factor of 2. In this case, the transform is specified via a statement. Transformation statements can be any valid Python code with the input/output passing through the transformation specified as `%x%`.

In [ ]:
tools.display_source_diff('yamls/connections_v0.yml', 'yamls/connections_transform_statement.yml', number_lines=True)

If you run this integration in the cell below, you can see that the resulting output file contains values that are double the values for the original integration.

In [ ]:
from yggdrasil.runner import run
run(['yamls/light_v0_python.yml', 'yamls/connections_transform_statement.yml'], production_run=True)
tools.display_source_diff('output/light_v0.txt', 'output/light_transform_statement.txt', number_lines=True)

Transformation that cannot be specified via a simple Python statement (e.g. if you need to use functions that are not builtins), then you can also pass the location of a Python function that can be loaded from a file via the `function` transformation parameter. The format should be `<filename>:<function name>`.

The YAML shown below performs the exact same transformation as the previous example, but is performed by the `double_light` function in the file `models/light_transform.py` (also displayed by the cell below).

In [ ]:
tools.display_source_diff('yamls/connections_v0.yml', 'yamls/connections_transform_function.yml', number_lines=True)
tools.display_source('models/light_transform.py', number_lines=True)

# Filtering Inputs & Outputs
Similarly to transforming inputs & outputs, sometimes modelers need to filter out some values (e.g. a threshold is reached where a different model needs to be used). For such cases, yggdrasil provides tools for excluding/including values passed through a connection.

The cell below shows the diff for a set of connections to the light model that excludes intensities from the light model if the are above 400 ergs/cm^2/s before passing them to the output file. In this case, the filter is specified via a statement. Filter statements are evaluated in the same way as transformation statements (can be any valid Python code with the input/output passing through the transformation specified as `%x%`), but should evaluate to a boolean (i.e. True or False).

In [ ]:
tools.display_source_diff('yamls/connections_v0.yml', 'yamls/connections_filter_statement.yml', number_lines=True)

If you run this integration in the cell below, you can see that the resulting output file does not contain the filtered values.

In [ ]:
run(['yamls/light_v0_python.yml', 'yamls/connections_filter_statement.yml'], production_run=True)
tools.display_source_diff('output/light_v0.txt', 'output/light_filter_statement.txt', number_lines=True)

Like transformations, filters can also be specified by passing the location of a Python function that can be loaded from a file via the `function` filter parameter. The format should be `<filename>:<function name>`.

The YAML shown below performs the exact same filter as the previous example, but is performed by the `filter_light` function in the file `models/light_filter.py` (also displayed by the cell below).

In [ ]:
tools.display_source_diff('yamls/connections_v0.yml', 'yamls/connections_filter_function.yml', number_lines=True)
tools.display_source('models/light_filter.py', number_lines=True)

## <span style="color:#C52060">Test your knowledge #8</span>

1. <span style="color:#C52060">Write a YAML that outputs intensities `<400` to one file and `>=400` to a different file.</span>
1. <span style="color:#C52060">Create a new version of the light model that calculates light using different parameters or a different algorithm and write a YAML that directs heights `>2` to the original model and `<=2` to the new model, but both models output to the same file.</span>
1. <span style="color:#C52060">Add a transformation to the output from the new model.</span>

## <span style="color:#C52060">Test your knowledge #9</span>

1. <span style="color:#C52060">Try running some of the other integrations via the CLI. What does the error look like if a YAML is missing?</span>
1. <span style="color:#C52060">Try running an integration with the `--debug` flag. What information does each log line include?</span>